In [29]:
# first neural network with keras tutorial
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [30]:
df = pd.read_pickle('../env_specs/prediction_data.pkl')
df.drop(['ProbeFileID','IsTarget'], axis=1, inplace=True)
# replace nans with -1
df = df.fillna(-1)

In [31]:
X,Y = df.drop(['label'], axis=1).values, df['label'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [32]:
def shuffle(arr):
    x, y = arr.shape
    rows = np.indices((x,y))[0]
    cols = [np.random.permutation(y) for _ in range(x)]
    return arr[rows, cols]

In [33]:
num_samples, num_algos = X_train.shape

del_orders = np.repeat(np.arange(num_algos).reshape(1, num_algos), repeats=num_samples, axis=0)#.repeat(num_algos-1, axis=0)

In [34]:
results = {}

def auc(y_true, y_pred):
                auc = tf.metrics.auc(y_true, y_pred)[1]
                K.get_session().run(tf.local_variables_initializer())
                return auc

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

h1, h2, h3 = 100, 400, 0

# define the keras model
model = Sequential()
model.add(Dense(h1, input_dim=X.shape[1], activation='relu'))
model.add(Dense(h2, input_dim=X.shape[1], activation='relu'))
if h3 > 0:
    model.add(Dense(h3, input_dim=X.shape[1], activation='relu'))
# model.add(Dense(50, input_dim=X.shape[1], activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auc])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [40]:
def del_algos(X, del_orders):
    for i in range(0, X.shape[0], num_algos-1):
        for j in range(num_algos-1):
            X[i+j, del_orders[i//(num_algos-1),:j+1]] = -1.
    return X

In [ ]:
epochs_per_shuffle = 5
X_train_rep = np.repeat(X_train, repeats=num_algos-1, axis=0)
Y_train_rep = np.repeat(Y_train, repeats=num_algos-1, axis=0)

for i in range(10):
    del_orders = shuffle(del_orders)
    X_train_del = del_algos(np.copy(X_train_rep), del_orders)
    # fit the keras model on the dataset
    model.fit(X_train_del, Y_train_rep, epochs=epochs_per_shuffle, validation_split=.10, batch_size=30, initial_epoch=i*5)

    # evaluate the keras model
#     _, accuracy, auc = model.evaluate(X_test, Y_test)
#     results['{},{},{}'.format(h1,h2,h3)] = [accuracy, auc]
#     print('h1: {}, h2: {}, h3: {}\nAccuracy: {:.4f}\nAUC: {:.4f}\n\n'.format(h1, h2, h3, accuracy, auc))

Instructions for updating:
Use tf.cast instead.
Train on 562582 samples, validate on 62510 samples
Epoch 1/5
543990/562582 [============================>.] - ETA: 1s - loss: 0.3956 - acc: 0.8150 - auc: 0.8330